# Section 1 - Differencial Privacy

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)]( https://colab.research.google.com/github/MarianoOG/Lesson-Notes-Secure-and-Private-AI/blob/master/Lectures/Section%201%20-%20Differencial%20Privacy.ipynb)

First of all, we talk about a **differencial atack** when an attaker is able to obtain information from a particular individual or group from a database by making a query to the entire database and to a sub-database derived from the original (with some data removed). Then, the attaker compares the difference between the outputs of this queries and with this results is able to obtain information about the individual or group. We say that a database is **differencially private** when this kind of attacks are not succesfull.

In this section we will first explore how a query can reveal information from an individual or group in the database. Then we will define differencial privacy with the help of the concept of sensitivity. And finally we will explore differencial privacy in the context of deep learning.

## Lesson 1: Simple Database Queries

We're going to make a toy database (for educational reasons) by initializing a random list of 1s and 0s which will represent sensitive data from a population. The number of entries directly corresponds to the number of people in our database.

**Key to the definition of differenital privacy is the ability to ask the question "When querying a database, if I removed someone from the database, would the output of the query be any different?"**. Thus, in order to check this, we will create sub-databases that we will call *parallel databases* which are databases with one entry removed.

In [1]:
'''
In this first project, we will create parallel databases "pdb" from the main database "db".
'''

# First we import the needed libraries
import torch

# Function to create database with n entries (number of people)
def create_db(n):
    return torch.rand(n) > 0.5

# Funtion to create 1 parallel database removing one particual index
def get_parallel_db(db, remove_index):
    return torch.cat((db[0:remove_index], 
                      db[remove_index+1:]))

# Funtion to create all parallel databases from a database
def get_parallel_dbs(db):
    parallel_dbs = list()
    for i in range(len(db)):
        pdb = get_parallel_db(db, i)
        parallel_dbs.append(pdb)
    return parallel_dbs

# Function to create database and parallel databases
def create_db_and_parallels(n):
    db = create_db(n)
    pdbs = get_parallel_dbs(db)
    return db, pdbs

# Creating actual database and parallel databases
db, pdbs = create_db_and_parallels(20)

# Printing results:
print(db)         # Database
print(len(pdbs))  # Number of parallel databases
# print(pdbs)      # Parallel databases

tensor([0, 0, 0, 1, 1, 0, 1, 0, 1, 0, 1, 0, 0, 1, 1, 1, 0, 1, 1, 0],
       dtype=torch.uint8)
20


## Lesson 2: Towards Evaluating The Differential Privacy of a Function

We want to be able to query our database and evaluate whether or not the result of the query is leaking private information. As mentioned previously, **this is about evaluating whether the output of a query changes when we remove someone from the database**. Specifically, we want to evaluate the *maximum* amount the query changes when someone is removed (maximum over all possible people who could be removed). 

In order to evaluate how much privacy is leaked, first we are going to define functions that will be our queries to the database. Then, we will measure the difference between each parallel db's query result and the query result for the entire database. Finally, we will calculate the maximum value along those results. This value is called sensitivity. **Sensitivity is measuring how much the output of the query changes when a person is removed from the database.**

In [2]:
'''
In this project we will:
1) Create a list of queries (sum, threshold and mean)
2) Calculate sensitivy for each one
'''
    
# Sum query function
def query_sum(db):
    return db.sum().float()

# Threshold query function
def query_threshold(db, threshold=5):
    return (db.sum() > threshold).float()

# Mean query function
def query_mean(db):
    return db.float().mean()

# Calculate sensitivity
def sensitivity(query, n):
    db, pdbs = create_db_and_parallels(n)
    db_result = query(db)
    max_distance = 0
    for pdb in pdbs:
        pdb_result = query(pdb)
        db_distance = torch.abs(pdb_result - db_result)
        if(db_distance > max_distance):
            max_distance = db_distance
    return max_distance

# Sensitivity for sum
print("Sum query")
for i in range(1,10):
    s = sensitivity(query_sum, i)
    print("N =", i, "- sentivity =", s)
print("Some results are not deterministic they will change if you re-run this cell.")
print("")
    
# Sensitivity for threshold
print("Treshold query")
for i in range(15):
    s = sensitivity(query_threshold, 10)
    print("N =", 10, "- sentivity =", s)
print("Even when the number of samples are the same the sensitity of this function varies")
print("")

# Sensitivity for mean
print("Mean query")
for i in range(1,10):
    s = sensitivity(query_mean, i*10)
    print("N =", i*10, "- sentivity =", s)
print("Observe how if the database is bigger the sensitivy is reduced.")

Sum query
N = 1 - sentivity = tensor(1.)
N = 2 - sentivity = tensor(1.)
N = 3 - sentivity = tensor(1.)
N = 4 - sentivity = tensor(1.)
N = 5 - sentivity = 0
N = 6 - sentivity = tensor(1.)
N = 7 - sentivity = tensor(1.)
N = 8 - sentivity = tensor(1.)
N = 9 - sentivity = tensor(1.)
Some results are not deterministic they will change if you re-run this cell.

Treshold query
N = 10 - sentivity = 0
N = 10 - sentivity = 0
N = 10 - sentivity = tensor(1.)
N = 10 - sentivity = 0
N = 10 - sentivity = 0
N = 10 - sentivity = 0
N = 10 - sentivity = 0
N = 10 - sentivity = 0
N = 10 - sentivity = 0
N = 10 - sentivity = 0
N = 10 - sentivity = 0
N = 10 - sentivity = 0
N = 10 - sentivity = 0
N = 10 - sentivity = 0
N = 10 - sentivity = 0
Even when the number of samples are the same the sensitity of this function varies

Mean query
N = 10 - sentivity = tensor(0.0667)
N = 20 - sentivity = tensor(0.0289)
N = 30 - sentivity = tensor(0.0230)
N = 40 - sentivity = tensor(0.0141)
N = 50 - sentivity = tensor(0.0122

## Lesson 3 - A Basic Differential Attack

Now we measure how sensible each function is for those particular cases but none of the functions we've looked at so far are differentially private. We will discover how to obtain information from them using a basic differential attack.

The most basic type of attack can be done as follows: let's say we wanted to figure out a specific person's value in the database. All we would have to do is query for the sum of the entire database and then the sum of the entire database without that person. Something similar will happen with the mean and threshold queries that we have.

In [3]:
'''
In this project we will:
1) Construct a database.
2) Demonstrate how two different sum queries can expose the value of the person represented by row 10.
'''

# Generate a database with only one parallel database (in index 10)
db = create_db(100)
pdb = get_parallel_db(db, remove_index=10)

# Printing reults:
print("Private value:", db[10])
print("")
print("Differencing attacks:")
print("\tSum query =", query_sum(db) - query_sum(pdb))
print("\tMean query =", query_mean(db) - query_mean(pdb))
print("\tThreshold query =", query_threshold(db, 50) - query_threshold(pdb, 50))
print("")
print("As you can see, the basic sum query is not differentially private at all!")

Private value: tensor(0, dtype=torch.uint8)

Differencing attacks:
	Sum query = tensor(0.)
	Mean query = tensor(-0.0058)
	Threshold query = tensor(0.)

As you can see, the basic sum query is not differentially private at all!


Differential privacy always requires a form of randomness added to the query. 

One technique is to add randomness to each person's response. Take in consideration that when you induce noise to each person's response you will obtain a skewed result. However, if the amount of noise introduced is known its posible to calculate an aproximation of the real result. It should be noted that, especially when we only have a few samples, this comes at the cost of accuracy. The greater the privacy protection the less accurate the results. 

In [4]:
'''
In this project we will:
1) Create a function that will introduce noise to the database 
2) Create a function that will calculate an aproximation of the real result from a skewed result.
2) Demostrate how the effect of the induced noise is diminished when the data set is bigger.
'''

def add_noise(db, noise=0.2):
    selected = (torch.rand(len(db)) > noise).float() # Will decide if the entry will be real or random
    random_answer = (torch.rand(len(db)) > 0.5).float()
    augmented_database = db.float() * selected + random_answer * (1 - selected)
    return augmented_database
    
def deskew(sk_result, noise=0.2):
    return ((sk_result / noise) - 0.5) * noise / (1 - noise)

for i in range(2,8):
    db = create_db(10**i)
    true_result = query_mean(db)
    augmented_database = add_noise(db)
    sk_result = query_mean(augmented_database)
    private_result = deskew(sk_result)
    print("N =", 10**i, "\tWithout Noise =", true_result, "\tWith Noise =", private_result)

N = 100 	Without Noise = tensor(0.5500) 	With Noise = tensor(0.5500)
N = 1000 	Without Noise = tensor(0.4720) 	With Noise = tensor(0.4575)
N = 10000 	Without Noise = tensor(0.4928) 	With Noise = tensor(0.4931)
N = 100000 	Without Noise = tensor(0.5005) 	With Noise = tensor(0.5009)
N = 1000000 	Without Noise = tensor(0.5005) 	With Noise = tensor(0.5011)
N = 10000000 	Without Noise = tensor(0.4999) 	With Noise = tensor(0.4998)


The previous method of adding noise was called **Local Differentail Privacy** because we added noise to each datapoint individually. This is necessary for some situations wherein the data is SO sensitive that individuals do not trust noise to be added later. However, it comes at a very high cost in terms of accuracy. 

Alternatively we can add noise _after_ data has been aggregated by a function. This kind of noise allows us to perform differential privacy on smaller groups of individuals with lower amounts of noise. Becasue of this accuracy will be less affected. However, participants must be able to trust that no-one looked at their datapoints _before_ the aggregation took place. This method is called **Global Differential Privacy**.

In [5]:
'''
In this project we will:
1) Create a function that will introduce noise to the result of the query (global differencial privacy)
'''

def M(true_result, noise=0.2):
    sk = (torch.rand(1)).float()*noise - noise/2.0
    private_result = true_result + true_result*sk
    return private_result

for i in range(1, 10):
    db = create_db(10*i)
    true_result = query_sum(db)
    private_result = M(true_result)
    print("N =", 10*i, "\tWithout Noise =", true_result, "\tWith Noise =", private_result)

N = 10 	Without Noise = tensor(4.) 	With Noise = tensor([4.0631])
N = 20 	Without Noise = tensor(9.) 	With Noise = tensor([9.2726])
N = 30 	Without Noise = tensor(13.) 	With Noise = tensor([12.6604])
N = 40 	Without Noise = tensor(27.) 	With Noise = tensor([25.1300])
N = 50 	Without Noise = tensor(28.) 	With Noise = tensor([27.5617])
N = 60 	Without Noise = tensor(29.) 	With Noise = tensor([27.7906])
N = 70 	Without Noise = tensor(32.) 	With Noise = tensor([34.4108])
N = 80 	Without Noise = tensor(42.) 	With Noise = tensor([38.2416])
N = 90 	Without Noise = tensor(55.) 	With Noise = tensor([52.0200])


## Lesson 4 - The Formal Definition of Differential Privacy

This definition is a measure of how much privacy is afforded by a query M. Specifically, it's a comparison between running the query M on a database (x) and a parallel database (y). As a reminder, parallel databases are defined to be the same as a full database (x) with one entry/person removed.

[![Image From: "The Algorithmic Foundations of Differential Privacy" - Cynthia Dwork and Aaron Roth](../Img/dp_formula.png)](https://www.cis.upenn.edu/~aaroth/Papers/privacybook.pdf)

This theorem is called "epsilon delta" differential privacy. It says: for all parallel databases, the maximum distance between a query on database (x) and the same query on database (y) will be e^epsilon plus a probability delta.

### Epsilon

**Epsilon Zero:** If a query satisfied this inequality with epsilon 0, then that would mean that the query for all parallel databases outputed the exact same value as the full database. If the sensitivity of a query is 0, then the epsilon value also happened to be zero.

**Epsilon One:** If a query satisfied this inequality with epsilon 1, then the maximum distance between the two random distributions M(x) and M(y) is 1.

### Delta

Delta is basically the probability that epsilon breaks. Sometimes the epsilon value is different for some queries than for others, delta represents this difference using probabilities. Note that this expression doesn't represent the full tradeoff between epsilon and delta.

## Lesson 5: How To Add Noise for Global Differential Privacy

In this lesson, we're going to learn about how to take a query and add varying amounts of noise so that it satisfies a certain degree of differential privacy (a certain epsilo-delta values). In particular, we're going to focus on global differential privacy.

There are two kinds of noise we can add: **Gaussian Noise** or **Laplacian Noise**. Generally speaking Laplacian is better, but both are still valid.

### How much noise should we add?

The amount of noise necessary to add to the output of a query is a function of four things:

- the type of noise (Gaussian/Laplacian)
- the sensitivity of the query/function
- the desired epsilon (ε)
- the desired delta (δ)

**Querying Repeatedly:** If we query the database multiple times we can simply add the epsilons, even if we change the amount of noise and their epsilons are not the same.

### Using Laplacian Noise

Laplacian noise is increased/decreased according to a _scale_ parameter b. We choose _b_ based on the following formula:

b = sensitivity(query) / epsilon

In other words, if we set b to be this value, then we know that we will have a privacy leakage of <= epsilon. 

Laplace function guarantees that delta value is equal to 0 (there are some tunings where we can have very low epsilon where delta is non-zero, but we'll ignore them for now).

In [6]:
'''
In this project we will:
1) Create a query function that adds the right amount of noise to satisfy an epsilon constraint.
2) Test it using a "sum" and "mean" query. Ensure that you use the correct sensitivity measures for both.
3) Use different epsilon values to determine its effect.
'''
import numpy as np

def laplacian_noise(sensitivity, epsilon):
    beta = sensitivity / epsilon
    noise = torch.tensor(np.random.laplace(0, beta, 1)).float()
    return noise

# Sum query
print("Sum query")
for i in range(1,5):
    epsilon = 1/(10**i)
    db = create_db(100)
    true_result = query_sum(db)
    noise = laplacian_noise(1, epsilon)
    private_result = true_result + noise
    print("N = 100\tEpsilon =", epsilon, "\tWithout Noise =", true_result, "\tWith Noise =", private_result)
print("")
    
# Mean query
print("Mean query")
for i in range(1,5):
    epsilon = 1/(10**i)
    db = create_db(100)
    true_result = query_sum(db)
    noise = laplacian_noise(1/100, epsilon)
    private_result = true_result + noise
    print("N = 100\tEpsilon =", epsilon, "\tWithout Noise =", true_result, "\tWith Noise =", private_result)

Sum query
N = 100	Epsilon = 0.1 	Without Noise = tensor(52.) 	With Noise = tensor([41.7960])
N = 100	Epsilon = 0.01 	Without Noise = tensor(45.) 	With Noise = tensor([67.4310])
N = 100	Epsilon = 0.001 	Without Noise = tensor(54.) 	With Noise = tensor([-482.8008])
N = 100	Epsilon = 0.0001 	Without Noise = tensor(44.) 	With Noise = tensor([-5588.0835])

Mean query
N = 100	Epsilon = 0.1 	Without Noise = tensor(55.) 	With Noise = tensor([55.0794])
N = 100	Epsilon = 0.01 	Without Noise = tensor(49.) 	With Noise = tensor([49.2251])
N = 100	Epsilon = 0.001 	Without Noise = tensor(54.) 	With Noise = tensor([55.5140])
N = 100	Epsilon = 0.0001 	Without Noise = tensor(45.) 	With Noise = tensor([82.1399])
